In [1]:
import json
import pprint

In [2]:
import os
git_dir = os.path.dirname(os.getcwd())
project_dir = os.path.join(git_dir,'zavmo')
os.chdir(project_dir)

In [3]:
import json
from helpers.chat import get_openai_completion,  get_prompt, make_structured_call, summarize_history, summarize_profile,create_message_payload,force_tool_call
from helpers.functions import parse_type
from IPython.display import Markdown
import openai
from openai import OpenAI
from helpers.functions import create_pydantic_model, get_yaml_data, create_system_message
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
stage_config = get_yaml_data('discuss')

In [37]:
resp_schema = get_yaml_data('common')['response']

In [38]:
resp_model    = create_pydantic_model(name=resp_schema['title'],
                                       description=resp_schema['description'],
                                       fields=resp_schema['fields']
                                       )
    
probe_system_message = create_system_message('discuss', stage_config, mode='probe')

In [11]:
required_fields = [f['title'] for f in stage_config['fields']]
required_fields

['interest_areas',
 'learning_style',
 'curriculum',
 'timeline',
 'goals_alignment']

In [28]:
profile_summary = """
Name: Mumtaz
Occupation: Junior Software Engineer

As a Junior Software Engineer, I am passionate about leveraging technology to solve complex problems and improve user experiences. I have a strong foundation in software development principles and a keen interest in expanding my knowledge and skills in various domains.

Learning Interests:
I am particularly interested in the following areas:

Statistics: Understanding data analysis techniques to derive meaningful insights.
Machine Learning (ML): Exploring algorithms and models that enable systems to learn from data.
Artificial Intelligence (AI): Delving into intelligent systems that mimic human behavior and decision-making.
Large Language Models (LLMs): Investigating advanced language processing technologies to enhance natural language understanding and generation.
I am eager to further develop my expertise in these fields and apply them to real-world applications, contributing to innovative projects and technologies."""

In [29]:
messages = []

In [30]:
user_input      = ''

In [31]:
probe_system_message = create_system_message("discuss", stage_config, mode='probe')
    
user_content         = f"""
    Here is what we know about the learner:
        {profile_summary}
           
    We need to probe the learner for the following fields: {required_fields}
    
    Learner's message: {user_input}
    """

In [ ]:
Markdown(probe_system_message)

In [32]:
message_payload = create_message_payload(user_content, 
                                             probe_system_message, 
                                             messages, 
                                             max_tokens=10000
                                             )

In [33]:
response_tool = force_tool_call(resp_model, 
                                    message_payload, 
                                    model='gpt-4o-mini',
                                    tool_choice='required',
                                    )

In [ ]:
assistant_message = response_tool.message
Markdown(assistant_message)

In [35]:
message_payload

[{'role': 'system',
  'content': "You are Zavmo, a highly interactive, supportive, and engaging AI assistant to help users learn and grow. Zavmo uses a 4-step learning process to help users learn and grow. The 4 Ds are Discover, Discuss, Deliver, and Demonstrate.\n\nBefore starting, you will need to know the user's profile details, this is the Profile Stage.\n\n# Guidelines\n\n- Friendly Engagement: Zavmo should adopt a warm, encouraging, and adaptable tone that reflects the user’s preferences and energy level, enhancing the enjoyment and reward of the learning experience.\n\n- Structured Process: Guide users through a 4-step learning process: Discover, Discuss, Deliver, and Demonstrate. This structured path keeps users oriented, with Zavmo ensuring smooth transitions between these stages.\n\n- Personalization: Tailor communication to individual user preferences, using their name and emojis to make the conversation more engaging and relatable.\n\n- Motivation & Feedback: Reinforce user

In [ ]:
messages.extend([{'role':'user','content':user_input},{'role':'assistant','content':assistant_message}])

In [5]:
messages=[{'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': "Hi Mumtaz! 🎉 We're excited to learn about your interests further. Are there any particular areas you're interested in exploring within those fields? 🎯 Also, how do you prefer to learn? (e.g., videos, reading, hands-on activities) 🎬📚🛠️"},
 {'role': 'user', 'content': 'no'},
 {'role': 'assistant',
  'content': "Alright, Mumtaz! Let's get a bit more specific on a few things to make sure we tailor your learning experience just right. 😃 \n\nAre there any specific areas within Statistics, Machine Learning, AI, or LLMs that you're most excited to dive into? 🌟\n\nAnd, what's your favorite way to learn? Do you prefer videos, reading, or maybe some hands-on projects? 🎬📚🛠️"},
 {'role': 'user', 'content': 'probability, machine learning, etc'},
 {'role': 'assistant',
  'content': "Great to hear you're interested in probability and machine learning, Mumtaz! 🎉 Let's make this learning experience even more exciting! How do you prefer to learn? (e.g., videos, reading, hands-on activities) 🎬📚🛠️"},
 {'role': 'user', 'content': 'handson'},
 {'role': 'assistant',
  'content': "Awesome! 🛠️ We'll include more hands-on activities in your plan. Now, based on our discussion, here are some possible modules for you: 1. Probability Theory 2. Machine Learning Basics 3. Advanced Machine Learning Techniques 4. Data Analysis with Statistics 5. Introduction to AI Systems 6. Working with LLMs. What do you think about these modules? 📚"},
 {'role': 'user', 'content': 'good'},
 {'role': 'assistant',
  'content': "Awesome! I’m glad you liked the modules! 🌟 Now, let's discuss the curriculum plan. Here’s a proposed structure:\n1. **Probability Theory**: Understanding foundational concepts.\n2. **Machine Learning Basics**: Introduction to key algorithms.\n3. **Advanced Machine Learning Techniques**: Exploring deeper methods and applications.\n4. **Data Analysis with Statistics**: Learning how to analyze data effectively.\n5. **Introduction to AI Systems**: Overview of AI technologies.\n6. **Working with LLMs**: Practical approaches to language models.\n\nDo you feel this plan aligns with your learning goals? Would you like to change or add anything? 🎯 Also, how quickly would you like to progress through the material? 🚀"},
 {'role': 'user', 'content': 'Its perfect i want to progress in 3 weeks'},
 {'role': 'assistant',
  'content': "Wonderful! 🌟 We'll set the timeline for completing your learning plan over 3 weeks!  Just to confirm, do you feel this plan aligns perfectly with your goals? 🎯 If there’s anything you’d like to adjust, let me know!"},
 {'role': 'user', 'content': 'okay'},
 {'role': 'assistant',
  'content': "Here's a summary of what we know about you, Mumtaz:\n\n- **Name**: Mumtaz\n- **Occupation**: Junior Software Engineer\n- **Learning Interests**:\n  - Statistics: Understanding data analysis techniques.\n  - Machine Learning (ML): Exploring algorithms and models.\n  - Artificial Intelligence (AI): Intelligent systems mimicking human behavior.\n  - Large Language Models (LLMs): Advanced language processing technologies.\n\n- **Learning Style**: Hands-on activities.\n- **Modules**: \n  1. Probability Theory  \n  2. Machine Learning Basics  \n  3. Advanced Machine Learning Techniques  \n  4. Data Analysis with Statistics  \n  5. Introduction to AI Systems  \n  6. Working with LLMs  \n\n- **Timeline**: 3 weeks.\n\nDoes everything look good? Would you like to make any updates or changes?"},
 {'role': 'user', 'content': 'No'},
 {'role': 'assistant',
  'content': "Thank you for sharing all that, Mumtaz! 🌟 We're all set to begin your personalized learning journey! I'm excited for you to dive into the world of Probability, Machine Learning, AI, and more with hands-on activities. Let's make this journey both enjoyable and impactful! 🚀 If you have any questions or need assistance along the way, feel free to ask. Let's get started!"}]

In [6]:
from helpers.tools.discuss import *

In [7]:
subjects = force_tool_call(GetCurriculum,messages).curriculum


In [9]:
subjects_list = [subject.dict() for subject in subjects]
subjects_list

[{'subject_name': 'Probability Theory',
  'modules': ['Basic Concepts of Probability',
   'Conditional Probability',
   "Bayes' Theorem",
   'Probability Distributions'],
  'duration': '1 week'},
 {'subject_name': 'Machine Learning Basics',
  'modules': ['Introduction to Machine Learning',
   'Supervised vs Unsupervised Learning',
   'Common Algorithms',
   'Model Evaluation Techniques'],
  'duration': '1 week'},
 {'subject_name': 'Advanced Machine Learning Techniques',
  'modules': ['Deep Learning Fundamentals',
   'Natural Language Processing',
   'Reinforcement Learning',
   'Model Optimization'],
  'duration': '1 week'},
 {'subject_name': 'Data Analysis with Statistics',
  'modules': ['Descriptive Statistics',
   'Inferential Statistics',
   'Hypothesis Testing',
   'Data Visualization Techniques'],
  'duration': '1 week'},
 {'subject_name': 'Introduction to AI Systems',
  'modules': ['AI Fundamentals',
   'Machine Learning vs AI',
   'Applications of AI',
   'Ethical Consideration

In [97]:
extract_system_message = create_system_message('discover', stage_config, mode='extract')
conversation_summary=summarize_history(messages)

In [98]:
user_message = f"""The learner is at the **Discover** stage.
    This is the (unverified) information we have about the learner:
        {profile_summary}

    Here is the summary of the conversation:
        {conversation_summary}
                    
    Which of the following fields can we add or update about the learner?
        {', '.join(required_fields)}
    """

In [99]:
message_payload = create_message_payload(user_message,  extract_system_message, [], max_tokens=10000)

In [100]:
######### Creating pydantic model for field having multiple properties ##########

# for field in stage_config['fields']:
#     if 'properties' in field:
#         create_pydantic_model(field['title'], field['properties'], field['description'])

In [101]:
stage_config = get_yaml_data('discuss')

In [79]:
models = [create_pydantic_model(field['title'], [field], field['description']) for field in stage_config['fields']]

In [80]:
# models = [create_pydantic_model(field['title'], field['properties'], field['description']) if 'properties' in field 
#           else create_pydantic_model(field['title'], [field], field['description']) 
#           for field in stage_config['fields']]


extraction_tools = force_tool_call(models,
                                       message_payload, 
                                       model='gpt-4o-mini',
                                       tool_choice='required',
                                       parallel_tool_calls=True)

In [ ]:
extraction_tools

In [103]:
Markdown(extract_system_message['content'])

You are an highly attentive assistant to Zavmo - a conversational AI assistant.

Zavmo will probe a learner for their profile details through a series of engaging and interactive questions. The learner will answer these questions to the best of their ability. 

Your job is to extract the information from the learner's answers, and build a profile for them, and present it to Zavmo. You will also guide Zavmo about what questions to ask next, or what actions to take.

At each interaction, you will be provided with the information we currently have about the learner, and the information that's still missing.

Your task is to extract the information from the learner's answers, and build a profile for them, and present it to Zavmo. You will also guide Zavmo about what actions to take next.

### Conversation Flow

Let's approach this step by step:

- First, review the information we currently have about the learner, and the information that's missing. You will be provided with a summary of information we have for the learner, and the information that's still missing. With this information, evaluate the learner's response to Zavmo's question.

- Next, If the learner responds with information that's relevant to some of the missing attributes, list down the new attributes that we can extract from the learner's response.

- If the learner's response is not relevant to any of the missing attributes, you should create a list with just "none".

- Finally, once you have a list of new attributes to extract, you will be provided with the appropriate schema for the attributes. Extract the information from the learner's response, and return it to Zavmo.


## On Completing the Stage

The stage is complete when all the fields are filled. When we have collected all the fields for the stage, the stage can be marked as complete, once the learner verifies the details you have collected.

- You should provide the details to Zavmo and ask Zavmo to present it to the learner so that they can verify the details.

If the learner verifies the details, you should ask Zavmo to finish the stage and provide a confirmation and encouragement to proceed to the next stage.

If the learner does not verify the details, you should ask Zavmo to present the learner with the details and ask them for corrections.

The learner is currently in the **Discover** stage. 

Here is a list of instructions for each attribue that we need to probe the learner for in the current stage:

**interest_areas**

Specific areas of interest the learner wants to focus on.

> Examples of learner responses and what we are extracting:
- I'm particularly interested in data visualization > interest_areas='Data visualization'
- Machine learning fascinates me > interest_areas='Machine learning'



**learning_style**

The preferred learning style of the learner (e.g., visual, auditory, kinesthetic).

> Examples of learner responses and what we are extracting:
- I learn best through hands-on projects > learning_style='Kinesthetic'
- Watching videos helps me understand better > learning_style='Visual'



**modules**

A list of modules or topics that will be included in the learner's personalized learning plan.


> Examples of learner responses and what we are extracting:
- Can we add more about data visualization? > modules='Include data visualization module'
- I'd prefer to focus more on advanced topics > modules='Emphasize advanced modules'



**curriculum**

A list of modules or topics designed for the learner's curriculum plan, detailing the focus areas and learning outcomes, including the main subject and title.


> Examples of learner responses and what we are extracting:
- Can we add more on data visualization? > modules='Include data visualization module'
- I'd prefer to focus more on advanced machine learning topics > modules='Emphasize advanced machine learning modules'



**timeline**

Propose a timeline for completing the learning plan.


> Examples of learner responses and what we are extracting:
- Can we make it shorter? > timeline='Shorter timeline preferred'
- I'd like a more extended timeline > timeline='Longer timeline preferred'



**goals_alignment**

Confirmation that the proposed plan aligns with the learner's goals.

> Examples of learner responses and what we are extracting:
- I'd like to spend less time on theory > goals_alignment='Reduce emphasis on theory'




In [ ]:
from pydantic import BaseModel


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = openai_client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [ ]:
# - field: prerequisites
  #   title: prerequisites
  #   annotation: list
  #   description: >
  #     The necessary qualifications or prior knowledge required for enrolling in the curriculum.
  #   probe_questions:
  #     - Do you have the necessary prerequisites for this curriculum? 📜
  #     - Would you like us to review any specific prerequisite knowledge? 🔍
  #   user_responses:
  #     - Yes, I meet all the prerequisites.
  #     - I might need a refresher on statistics.
  #   extract_examples:
  #     - "I might need a refresher on statistics > prerequisites='Needs refresher on statistics'"
  #     - "Yes, I meet all the prerequisites > prerequisites='Qualified to enroll'"
  #   probe_responses:
  #     - Great! You're ready to start. 🚀
  #     - No problem! We can provide resources for statistics. 📘

In [34]:
def parse_curriculum(curriculum_json):
    try:
        # Load the curriculum JSON
        curriculum_data = json.loads(curriculum_json)

        # Validate that the input is a list
        if not isinstance(curriculum_data, list) or not curriculum_data:
            raise ValueError("Expected a non-empty list of curriculums.")

        # Extract relevant fields for the first curriculum
        curriculum = curriculum_data[0]  # Assuming we want to process the first curriculum
        parsed_data = {
            "title": curriculum.get('title', '').strip(),
            "subject": curriculum.get('subject', '').strip(),
            "level": str(curriculum.get('level', '')).strip(),
            "modules": []
        }

        # Validate main curriculum fields
        if not parsed_data["title"]:
            raise ValueError("Curriculum title is required.")
        if not parsed_data["subject"]:
            raise ValueError("Curriculum subject is required.")
        if not parsed_data["level"].isdigit():
            raise ValueError("Curriculum level must be a numeric value.")

        # Loop through modules and extract information
        for module in curriculum.get('modules', []):
            module_info = {
                "module_title": module.get('title', '').strip(),
                "lessons": [],
                "learning_outcomes": []
            }

            # Validate module fields
            if not module_info["module_title"]:
                raise ValueError("Module title is required.")

            # Extract lessons within the module
            for lesson in module.get('lessons', []):
                lesson_title = lesson.get('title', '').strip()
                content = lesson.get('content', '').strip()
                duration = lesson.get('duration', 0)

                if not lesson_title:
                    raise ValueError("Lesson title is required.")
                if not content:
                    raise ValueError("Lesson content is required.")
                if not isinstance(duration, (int, float)) or duration <= 0:
                    raise ValueError(f"Lesson duration must be a positive number. Got: {duration}")

                module_info["lessons"].append({
                    "lesson_title": lesson_title,
                    "content": content,
                    "duration": duration
                })

            # Extract learning outcomes and assessment criteria
            for outcome in module.get('learning_outcomes', []):
                description = outcome.get('description', '').strip()
                assessment_criteria = outcome.get('assessment_criteria', [])

                if not description:
                    raise ValueError("Learning outcome description is required.")
                
                # Validate assessment criteria
                if not isinstance(assessment_criteria, list):
                    raise ValueError("Assessment criteria must be a list.")
                assessment_criteria = [criterion.strip() for criterion in assessment_criteria if criterion.strip()]

                module_info["learning_outcomes"].append({
                    "description": description,
                    "assessment_criteria": assessment_criteria
                })

            parsed_data["modules"].append(module_info)

        return parsed_data

    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None
    except ValueError as ve:
        print(f"Validation error: {ve}")
        return None


In [35]:

curriculum_json = '''
[
 {
   "title": "Level 5 Diploma in Business Management",
   "subject": "Business Management",
   "level": "5",
   "modules": [
     {
       "title": "Organizational Behavior",
       "learning_outcomes": [
         {
           "description": "Understand and analyze organizational behavior concepts",
           "assessment_criteria": [
             "Explain key theories of organizational behavior",
             "Analyze the impact of organizational culture on performance"
           ]
         }
       ],
       "lessons": [
         {
           "title": "Introduction to Organizational Behavior",
           "content": "This lesson introduces the key concepts of organizational behavior...",
           "duration": 120
         },
         {
           "title": "Individual Behavior in Organizations",
           "content": "Learn about personality, perception, and individual decision making...",
           "duration": 120
         }
       ],
       "duration": 60
     }
   ],
   "prerequisites": ["Level 4 qualification in a related subject"],
   "qualification_level": 5,
   "guided_learning_hours": 240,
   "total_qualification_time": 480,
   "assessment_methods": ["Written examination", "Case study analysis", "Research project"]
 }
]
'''

In [36]:
parsed_curriculum = parse_curriculum(curriculum_json)
pprint.pprint(parsed_curriculum)


{'level': '5',
 'modules': [{'learning_outcomes': [{'assessment_criteria': ['Explain key '
                                                             'theories of '
                                                             'organizational '
                                                             'behavior',
                                                             'Analyze the '
                                                             'impact of '
                                                             'organizational '
                                                             'culture on '
                                                             'performance'],
                                     'description': 'Understand and analyze '
                                                    'organizational behavior '
                                                    'concepts'}],
              'lessons': [{'content': 'This lesson introduces the key concept